In [1]:
from utils import QA

In [2]:
prompt = 'What does the customer need to provide at the time of placing an order?'
QA(prompt)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at models/QA_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


('updated information and documents', '0.90')